In [40]:
import geopandas as gpd
from geopandas.geoseries import *
import numpy as np

# 190 PLZs
plz = gpd.read_file('plz.geojson')

#447 PLRs
plr = gpd.read_file('berlin-lor.planungsraeume.geojson')

df = pd.DataFrame(columns = ['PLR', 'PLR_NAME', 'PLZ'])
df['PLZ'] = np.empty((len(plr), 0)).tolist()

for indexPLR, rowPLR in plr.iterrows():
    df.loc[indexPLR, 'PLR'] = rowPLR.SCHLUESSEL[1:] if rowPLR.SCHLUESSEL.startswith('0') else rowPLR.SCHLUESSEL
    df.loc[indexPLR, 'PLR_NAME'] = rowPLR.PLR_NAME
    for indexPLZ, rowPLZ in plz.iterrows():
        if rowPLZ.geometry.intersects(rowPLR.geometry):
            df.loc[indexPLR, 'PLZ'].append(str(rowPLZ.spatial_name))

df.to_csv('PLR2PLZ.csv')


df2 = pd.DataFrame(columns = ['PLZ', 'PLR'])
df2['PLR'] = np.empty((len(plz), 0)).tolist()

for indexPLZ, rowPLZ in plz.iterrows():
    df2.loc[indexPLZ, 'PLZ'] = rowPLZ.spatial_name
    for indexPLR, rowPLR in plr.iterrows():
        if rowPLR.geometry.intersects(rowPLZ.geometry):
            df2.loc[indexPLZ, 'PLR'].append(str(rowPLR.SCHLUESSEL[1:] if rowPLR.SCHLUESSEL.startswith('0') else rowPLR.SCHLUESSEL))

df2.to_csv('PLZ2PLR.csv')
